# Pràctica 7. Ús de bucles condicionals i funcions amb Tkinter. Tres en ratlla
#### Adrià Rojo
---

Defineixo funcions per dibuixar figures des del seu centre

In [2]:
# importo tkinter
import tkinter

# defineixo funcions útils
def crear_cercle(canvas: tkinter.Canvas, x: int, y: int, radi: int, **kwargs) -> None:
    """
    Crea un cercle a les coordenades x i y, amb el radi asignat, al canvas passat per arguments
    """
    canvas.create_oval(x-radi, y-radi, x+radi, y+radi, **kwargs)

def crear_rectangle(canvas: tkinter.Canvas, x: int, y: int, llargada: int, altura: int, **kwargs) -> None:
    """
    Crea un rectangle amb el centre a x, y i de llargada i altura asignats
    """
    canvas.create_rectangle(x-llargada/2, y-altura/2, x+llargada/2, y+altura/2, **kwargs)

def crear_quadrat(canvas: tkinter.Canvas, x: int, y: int, costat: int, **kwargs) -> None:
    """
    Crea un quadrat amb el centre a x, y i de costat asignats
    """
    canvas.create_rectangle(x-costat/2, y-costat/2, x+costat/2, y+costat/2, **kwargs)


## Pas 1: Funció demanar un enter

In [3]:
def demanar_enter(text: str, min: int = 1, max: int = 3) -> int:
    """
    Demana una entrada numérica a l'usuari amb un text i que estigui dins del rang [min, max]
    """
    entrada = ""
    while (not entrada.isdigit()) or (not min <= int(entrada) <= max): 
        # degut al funcionament de l'or, puc comprovar al principi si es un digit primerament
        # i quan passa aquesta condició, veure si està dins del rang
        entrada = input(text)

    return int(entrada)


## Pas 2: Inicialitzar finestra

In [4]:
CANVAS_WIDTH = 600
CANVAS_HEIGHT = 600

# res a comentar
def ini_finestra() -> tkinter.Canvas:
    """
    Inicialitza una finestra blanca 600x600 (CANVAS_WIDTHxCANVAS_HEIGHT) i retorna el root i el canvas
    """
    root = tkinter.Tk()
    root.title("3 en ratlla")
    canvas = tkinter.Canvas(root,
                            background='white',
                            height=CANVAS_WIDTH,
                            width=CANVAS_HEIGHT)
    canvas.pack()
    return canvas

## Pas 3: Inicialitzar graella


In [5]:
SIDE_SQUARE = 150
OFFSET_TOP = 75
OFFSET_LEFT = 75

def ini_graella(finestra: tkinter.Canvas, **kwargs) -> None:
    """
    Dibuixa la graella amb quadrats de costat SIDE_SQUARE i amb un offset a cada costat de OFFSET_TOP i OFFSET_LEFT
    """
    # en comptes de fer 4 linies i columens, faig 3x3 rectangles (que es le matiex)
    # li afegeixo l'offset de cada costat i mig rectangle per què dibuixo les figures des del seu centre
    for i in range(3):
        for j in range(3):
            crear_quadrat(finestra, OFFSET_LEFT + SIDE_SQUARE/2 + SIDE_SQUARE * i, OFFSET_TOP + SIDE_SQUARE/2 + SIDE_SQUARE * j, SIDE_SQUARE, **kwargs)


## Pas 4: Marcar casella

In [6]:
PLAYER_1 = 1
PLAYER_2 = 2

def marca_casella(finestra: tkinter.Canvas, columna: int, fila: int, jugador: int) -> None:
    """
    Marca la casella a la fila i la columna asignades pel jugador
    """
    # El calcul es per la mateixa raó que a ini_graella, com la fila i la columna estàn dins del rang [1, 3] he de restar 1 per
    # poder-ho dibuixar a dins de la graella
    crear_cercle(finestra, OFFSET_LEFT + SIDE_SQUARE/2 + SIDE_SQUARE*(columna-1), OFFSET_TOP+SIDE_SQUARE/2 +SIDE_SQUARE*(fila-1), SIDE_SQUARE/2, fill=("red" if jugador == PLAYER_1 else "blue"))



## Pas 5: Control del ratolí

In [7]:
# Funció que s'activa quan es clica el botó esquerre del ratolí
def motion(event):
    """Aquesta funció està preparada per rebre un event de TKinter.Quan es rep l'event s'imprimeixen les coordenades del clica la finestra."""
    print("Mouse position: (%s %s)" % (event.x, event.y))

# Inicialitzem TKinter
master = tkinter.Tk()
# Posem un missatge de text a la finestra
missatge = "Cliqueu amb el ratolí sobre qualsevol punt de l'area verda. Observeu que s'imprimeixen les coordenades del clic."
msg = tkinter.Message(master, text = missatge)
msg.config(bg='white', font=('times', 24, 'italic'))
msg.pack()

# Inicialitzem el canvas
canvas = tkinter.Canvas(master, borderwidth = 0,background = 'green',height = 300, width = 300)
# En la zona del canvas associem el click esquerra del
# ratolí a la funció "motion"
canvas.bind('<ButtonRelease-1>',motion)
canvas.pack()
# Bucle principal de TKinter. Tanqueu la finestra per acabar.
tkinter.mainloop()

## Pas 6: Control bàsic del joc

In [9]:
# finestres emergents
import tkinter.messagebox

# definir variables globals
jugades = 0
acabat = False
game = [
    [None, None, None],
    [None, None, None],
    [None, None, None]
]
current_player = PLAYER_1

def check_guanyador() -> bool:
    """
    Comprova la variable game i si hi ha un n-en ratlla retorna True, en cas contrari retorna None
    """
    # Defineixo funcio helper que donat un enumerador returna si tots els elements del mateix son iguals al current_player
    def check_linea(linea) -> bool:
        return all(x == current_player for x in linea)
 
    for i in range(len(game)): 
        if (check_linea(game[i]) or check_linea(linea[i] for linea in game)): return True # Miro per cada fila (game[i]) i per cada columna (game[?][i])
    
    #      miro per la diagonal (game[i][i])                            
    #                 |         miro per la diagonal inversa aprofitant que es pot accedir a l'array des del final (game[i][-i-1]) (per exemple game[0][-1])
    #                 |                                                                         |
    #                 V                                                                         V
    return (check_linea(linea[ind] for (ind, linea) in enumerate(game)) or check_linea(linea[-(ind+1)] for (ind, linea) in enumerate(game)))

def control_joc(event: tkinter.Event) -> None:
    """
    Executa la logica del joc cada vegada que hi ha un click
    """
    # variables globals (array no ja que es de referencia)
    global current_player 
    global jugades
    global acabat
    
    # si s'ha acabat talla el joc
    if (acabat):
        return

    # agafo la columna i la fila corresponent al ratolí (com veure a quina secció   )
    x = int((event.x - OFFSET_LEFT) // SIDE_SQUARE) +1
    y = int((event.y - OFFSET_TOP) // SIDE_SQUARE) +1

    
    # check si el click ha caigut fora de la graella
    if not (1 <= x <= len(game) and 1 <= y <= len(game)):
        print("Fora de linea")
        return
        
    # check si la casella esta ocupada
    if (game[y-1][x-1] != None):
        print("Casella ocupada")
        return

    # tot be: augmentem jugades, asignem casella al jugador i pintem fitxa
    jugades += 1
    game[y-1][x-1] = current_player
    marca_casella(event.widget, x, y, current_player)
    
    # check el guanyador a partir de la jugada 5 (abans no te sentit)
    if (jugades >= (len(game) * 2 -1) and check_guanyador()):
        acabat = True
        tkinter.messagebox.showinfo("Guanyador!", message="Guanyador jugador " + str(current_player))
        return

    current_player = PLAYER_2 if current_player == PLAYER_1 else PLAYER_1 
    # check de si s'ha acabat el joc
    if (jugades == 9):
        acabat = True
        tkinter.messagebox.showinfo("Fi", message="Fi del joc!")

finestra = ini_finestra()
ini_graella(finestra)

finestra.bind('<ButtonRelease-1>',control_joc)

finestra.mainloop()   


## Pas 7: Millores
Totes les millores han siguit implementades amb un parell de canvis:
* La graella que controla l'estat del joc (variable `game`) marca amb `None` en comptes de `0` quan la casella esta buïda
* Faig el check de que una casella està plena
* Implemento la funcio `check_guanyador()` per a un joc en ratlla genèric (n-en ratlla) (possiblement hi hagin solucions més fàcils de llegir amb la funció `all()`)
---
### Explicació del flux de l'exercici

Tinc diverses variables de l'estat del joc:

* El jugador actual: `current_player`
* Quantitat de jugades: `jugades`
* Partida acabada: `acabat` de forma que `True` si s'ha acabat o `False` si no
* Estat de les caselles: `game`
    * De forma que es pot interpretar d'aquesta forma:
    ```
    |---|---|---|           [
    | A | B | C |               [A, B, C],
    |---|---|---|
    | D | E | F |               [D, E, F],
    |---|---|---|
    | G | H | I |               [G, H, I]
    |---|---|---|           ]

    Graella a pantalla      Graella en programació / memòria, implementada amb una llista de llistes
                            on hi ha una llista superior que conté fileres, i cada filera és una llista
                            amb caselles
    ```